In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)
# 데이터 출처 - https://www.nie-ecobank.kr/rdm/rsrchdoi/selectRsrchDtaListVw.do (에코뱅크(국제생태정보종합은행))

In [27]:
pd.read_csv("전국자연환경조사_조류_2014.csv")

,id,학명,한글보통명,위도,경도,조사일자,조사연도,시도명,개체수,geometry
0,8000000002364164,Podiceps cristatus,뿔논병아리,36.733111,126.654500,2014-03-14,2014,충청남도,NaN,POINT (169139.2620999999 459450.5734999999)
1,8000000002364165,Podiceps cristatus,뿔논병아리,36.733111,126.654500,2014-03-14,2014,충청남도,NaN,POINT (169139.2620999999 459450.5734999999)
2,8000000002097468,Dendrocopos major,오색딱다구리,36.655028,126.638667,2014-03-15,2014,충청남도,NaN,POINT (167692.3596000001 450790.7155000009)
3,8000000002097453,Buteo buteo,말똥가리,36.640000,126.694306,2014-03-15,2014,충청남도,NaN,POINT (172661.8948999997 449105.7410000004)
4,8000000002097451,Picus canus,청딱다구리,36.636556,126.686917,2014-03-15,2014,충청남도,NaN,POINT (171999.8331000004 448725.6973000001)
...,...,...,...,...,...,...,...,...,...,...
9388,8000000002276849,Dendrocopos major,오색딱다구리,37.953536,127.046392,2014-10-30,2014,경기도,NaN,POINT (204077.2844000002 594843.7075999994)
9389,8000000002278839,Dendrocopos major,오색딱다구리,35.653889,129.073944,2014-10-30,2014,울산광역시,NaN,POINT (387821.3569 341623.5326000005)
9390,8000000002301998,Buteo buteo,말똥가리,37.950919,127.122331,2014-10-31,2014,경기도,NaN,POINT (210751.8006999996 594559.2513999995)
9391,8000000002302007,Dendrocopos major,오색딱다구리,37.973867,127.072175,2014-10-31,2014,경기도,NaN,POINT (206341.5571999997 597101.7865999993)


In [ ]:
# https://sgis.kostat.go.kr/view/pss/requstDataListDtl 통계지리정보 서비스에 자료신청함(2024-01-12)
# https://egis.me.go.kr/ 여기 데이터 찾아보기